<a href="https://colab.research.google.com/github/jungry/time_series/blob/main/TimeSeries_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Week 4 - Assignment

## 1. Modeling Seasonal Variations

### 1.1. Visualization for Item Sales Data
**Task:** Visualize the item sales data to identify trends and seasonal variations


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load the sales dataset and understand its characteristics
!pip install yfinance


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.stats.stattools import durbin_watson
import yfinance as yf
!ls /content/drive/Shared\ drives/

24_Fall_TimeSeriesAnalysis_Project  timeseries-datasets


In [4]:
# Ensure the Dates are in datetime format and use the column as an index in your dateset


In [5]:
# Define the target variable (y)

# Split the dataset to training and test set (leave 2 years for testing)


In [6]:
# Plot the training data and analyze the visual trends and seasonality


### 1.2. Trend Linear Modeling
**Task:** Fit a linear model to the sales data to capture the overall trend

In [7]:
# Define the feature (X) and target (y) variables that you will use to plot the trend line

# Fit and predict the model on the features

# Plot the actual sales data and the fitted linear trend


### 1.3. Seasonality Modeling - Binary Variable Models
**Task:** Model seasonality using binary (dummy) variables

In [8]:
# Create a separate dataset for binary seasonality modeling

# Define the time index (t), constant and extract month variable from the data

# Create month dummy variables

# Remove the January column

# Display the information of the columns


In [9]:
# Split the data into features(X) and target variable (y)


In [10]:
# Fit a linear regression model that includes both the trend (time) and binary variables to model seasonality

# Output regression coefficients


In [11]:
# Plot the results and compare with the raw data


### 1.4. Seasonality Modeling - Trigonometric Models
**Task:** Model seasonality using trigonometric functions (sine and cosine)

In [12]:
# Create a separate dataset for trigonometric seasonality modeling

# Define the time index (t) and constant

# Define the period of seasonality (L)


In [13]:
# Split the data into features(X) and target variable (y)


In [14]:
# Fit a linear regression model that includes both the trend (time) and binary variables to model seasonality

# Output regression coefficients


In [15]:
# Plot the results and compare with the raw data


### 1.5. Durbin-Watson Test for Autocorrelation Check for Residuals
**Task:** Check for autocorrelation in the residuals using the Durbin-Watson test

In [16]:
# Calculate the residuals of the actual values vs the two seasonal components (Binary and Trigonometric)

# Calculate Durbin-Watson test statistic


### 1.6 Evaluation on Test Dataset
**Task:** Repeat the above tasks on test dataset and evaluate the models created for the seasonality components using MSE

In [17]:
# Create a separate dataset for binary seasonality modeling

# Define the time index (t), constant and extract month variable from the data

# Create month dummy variables

# Remove the January column


In [18]:
# Split the data into features(X) and target variable (y)


In [19]:
# Create a separate dataset for trigonometric seasonality modeling

# Define the time index (t), constant and extract month variable from the data

# Define the period of seasonality (L)


In [20]:
# Split the data into features(X) and target variable (y)


In [21]:
# Evaluate the two models using Mean Squared Error (MSE) on the test dataset


## 2. Modeling Growth Curve Models

### 2.1. Prepare the Data
**Task:** Prepare the data for growth curve modeling

In [22]:
given_data = {
    "Year": np.arange(1, 16),
    "Online_Courses": [15, 20, 25, 35, 45, 60, 80, 110, 145, 185, 230, 300, 475, 525, 750]
}
# Create a dataframe from the given data

# Plot the raw data to visualize growth trends


### 2.2. Transform the Data Using Logarithm
**Task:** Apply a logarithmic transformation to the sales data

In [23]:
# Transform the Sales column using the natural logarithm (np.log)

# Create a figure with two subplots side by side

# Plot the raw sales data on the first subplot

# Plot the log-transformed sales data on the second subplot

# Adjust layout to avoid overlap


### 2.3. Fit a Linear Regression Model to the Transformed Data
**Task:** Fit a linear regression model to the logarithmically transformed sales data

In [24]:
# Use the time variable as the predictor and the log-transformed sales as the target

# Fit the model and plot the predicted values alongside the log-transformed sales


### 2.4. Fit a Linear Regression Model to the Raw Data
**Task:** Fit a linear regression model to the raw sales data

In [25]:
# Use the same time variable and fit a model to the raw sales data

# Plot the predicted values alongside the actual raw sales data


### 2.5. Visualize and Evaluate the Model
**Task:** Evaluate the performance of both models (on transformed and raw data)

In [26]:
# Predict values using the raw model

# Predict values using the log model and convert log predictions back to the original scale

# Plot the actual vs predicted values


In [27]:
# Calculate the error metrics such as Mean Absolute Error (MAE) and Mean Squared Error (MSE) for both models

# Compare which model performs better based on the error values